In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")


import pingouin as pg
import scikit_posthocs as sp


csv_path = "policy_comparison_by_key_combined_all10.csv"  
policies = ["accept_else_bestp","accept_if_premium","always_accept","random_uniform","value_net"]
outcome_name = "score"  


df = pd.read_csv(csv_path)


df["key"] = (
    df["seed"].astype(str) + " | " +
    df["start_date"].astype(str) + " | " +
    df["start_time"].astype(str) + " | " +
    df["start_location"].astype(str)
)


long = df.melt(
    id_vars=["key"],
    value_vars=policies,
    var_name="policy",
    value_name=outcome_name
).dropna()


desc = long.groupby("policy")[outcome_name].agg(
    n="count", mean="mean", sd="std", median="median",
    q25=lambda x: np.percentile(x,25),
    q75=lambda x: np.percentile(x,75)
).reset_index()
print("Per-policy descriptives:\n", desc, "\n")


model = smf.mixedlm(f"{outcome_name} ~ C(policy)", data=long, groups=long["key"]).fit()
resid = model.resid


k2_stat, k2_p = stats.normaltest(resid)  
shapiro_p = stats.shapiro(resid).pvalue 
print(f"Residual normality: K² p={k2_p:.4g}" + (f" | Shapiro p={shapiro_p:.4g}" if not np.isnan(shapiro_p) else ""))


policies = ["accept_else_bestp","accept_if_premium","always_accept","random_uniform","value_net"]

wide = (
    df.assign(key=df["seed"].astype(str)+" | "+df["start_date"].astype(str)+" | "+df["start_time"].astype(str)+" | "+df["start_location"].astype(str))
      .pivot_table(index="key", values=policies, aggfunc="first")[policies]
      .apply(pd.to_numeric, errors="coerce")
      .dropna(how="any")
)
print(wide.shape, wide.dtypes) 

spher, W, chisq, dof, pval = pg.sphericity(wide)
print(spher, round(W, 3), round(chisq, 3), dof, pval)

eps = pg.epsilon(wide)        
print(eps)


Per-policy descriptives:
               policy    n        mean         sd      median         q25  \
0  accept_else_bestp  250  243.651551  42.011885  242.525916  206.675055   
1  accept_if_premium  250  238.196821  45.611127  241.855051  194.706748   
2      always_accept  250  241.668988  42.198539  241.078803  204.856883   
3     random_uniform  250  139.534814  22.666887  137.191662  123.642498   
4          value_net  250  262.240336  35.522934  260.290842  235.871923   

          q75  
0  283.373039  
1  278.294231  
2  274.710934  
3  153.505975  
4  290.445630   

Residual normality: K² p=0.4783 | Shapiro p=0.4243
(250, 5) accept_else_bestp    float64
accept_if_premium    float64
always_accept        float64
random_uniform       float64
value_net            float64
dtype: object
False 0.388 234.201 9 2.1810973644203323e-45
0.6461177800863207


In [ ]:
from statsmodels.stats.anova import AnovaRM
from scipy.stats import f
import numpy as np
import pandas as pd


anova_df = long.rename(columns={"key":"subject","policy":"within","score":"dv"})
fit = AnovaRM(anova_df, depvar="dv", subject="subject", within=["within"]).fit()

F = float(fit.anova_table["F Value"][0])
k = wide.shape[1]              
n = wide.shape[0]               


df1_gg = eps * (k - 1)
df2_gg = eps * (k - 1) * (n - 1)
p_gg = f.sf(F, df1_gg, df2_gg)

print(f"RM-ANOVA (GG-corrected): F({df1_gg:.2f}, {df2_gg:.2f})={F:.2f}, p={p_gg:.3e}")


pg.rm_anova(wide)

RM-ANOVA (GG-corrected): F(2.58, 643.53)=1475.68, p=2.052e-270


,Source,ddof1,ddof2,F,p-unc,p-GG-corr,ng2,eps,sphericity,W-spher,p-spher
0,Within,4,996,1475.67523,0.0,2.052477e-270,0.562825,0.646118,False,0.388063,2.181097e-45


In [3]:
pg.pairwise_ttests(dv="score", within="policy", subject="key",
                        data=long, padjust="holm", effsize="cohen")


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,policy,accept_else_bestp,accept_if_premium,True,True,4.809843,249.0,two-sided,2.617825e-06,7.853474e-06,holm,3995.026,0.124400
1,policy,accept_else_bestp,always_accept,True,True,1.507054,249.0,two-sided,1.330644e-01,1.330644e-01,holm,0.217,0.047086
2,policy,accept_else_bestp,random_uniform,True,True,46.722979,249.0,two-sided,3.163726e-125,2.847353e-124,holm,2.044e+121,3.084493
3,policy,accept_else_bestp,value_net,True,True,-12.259982,249.0,two-sided,2.381395e-27,9.525580e-27,holm,1.416e+24,-0.477825
4,policy,accept_if_premium,always_accept,True,True,-2.576908,249.0,two-sided,1.054556e-02,2.109112e-02,holm,1.808,-0.079024
5,policy,accept_if_premium,random_uniform,True,True,40.661423,249.0,two-sided,6.147942e-112,4.303559e-111,holm,1.227e+108,2.739469
6,policy,accept_if_premium,value_net,True,True,-14.130048,249.0,two-sided,1.097506e-33,6.585035e-33,holm,2.545e+30,-0.588157
7,policy,always_accept,random_uniform,True,True,45.517759,249.0,two-sided,1.074528e-122,8.596226e-122,holm,6.193e+118,3.015377
8,policy,always_accept,value_net,True,True,-13.276016,249.0,two-sided,8.964330e-31,4.482165e-30,holm,3.384e+27,-0.527419
9,policy,random_uniform,value_net,True,True,-62.596490,249.0,two-sided,2.351293e-154,2.351293e-153,holm,2.008e+150,-4.118114


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")
import pingouin as pg
import scikit_posthocs as sp


csv_path = "policy_comparison_by_key_combined_all10.csv"
policies = ["accept_else_bestp","accept_if_premium","always_accept","random_uniform","value_net"]
outcome_name = "score"


df = pd.read_csv(csv_path)


for p in policies:
    df[p] = pd.to_numeric(df[p], errors="coerce")


df["scenario"] = (
    df["start_date"].astype(str) + " | " +
    df["start_time"].astype(str) + " | " +
    df["start_location"].astype(str)
)


per_scenario = (
    df.groupby("scenario", as_index=False)
      .agg(**{p: (p, "mean") for p in policies}, n_seeds=("seed", "nunique"))
)


agg_dict = {"n_seeds": ("seed", "nunique")}
for p in policies:
    agg_dict.update({
        f"{p}_mean": (p, "mean"),
        f"{p}_std":  (p, "std"),
    })

per_scenario = (
    df.groupby("scenario", as_index=False)
      .agg(**agg_dict)
)




ordered_cols = (
    ["scenario", "n_seeds"] +
    [col for p in policies for col in (f"{p}_mean", f"{p}_std")]
)
per_scenario = per_scenario[ordered_cols]


print(per_scenario.to_string(index=False, float_format=lambda x: f"{x:,.3f}"))


tidy = (
    per_scenario
      .set_index(["scenario", "n_seeds"])
      .rename(columns=lambda c: c.replace("_mean","|mean")
                                 .replace("_std","|std"))
)

tidy = (
    tidy
      .stack()                              # stacks the policy|stat columns
      .rename("value")
      .reset_index()
)

tidy[["policy","stat"]] = tidy["level_2"].str.split("|", n=1, expand=True)
tidy = tidy.drop(columns=["level_2"]).pivot(index=["scenario","n_seeds","policy"], columns="stat", values="value").reset_index()

print(tidy.to_string(index=False, float_format=lambda x: f"{x:,.3f}"))





                                      scenario  n_seeds  accept_else_bestp_mean  accept_else_bestp_std  accept_if_premium_mean  accept_if_premium_std  always_accept_mean  always_accept_std  random_uniform_mean  random_uniform_std  value_net_mean  value_net_std
2017-12-01 | 11:00 | (51.53342819,-0.05728552)       25                 197.203                 11.259                 187.926                 17.559             200.071             11.766              128.466              14.939         232.441         20.907
2019-06-15 | 08:45 | (51.50913620,-0.06112576)       25                 238.915                 12.322                 236.459                 13.733             243.335             11.931              124.688              15.248         257.701         14.272
2020-02-21 | 10:30 | (51.50685120,-0.00980000)       25                 194.396                 13.394                 183.880                 13.746             193.357             14.420              122.696        